In [1]:
# Import modules
import pandas as pd
import numpy as np
import re
from functools import reduce

# Set pandas number format
pd.options.display.float_format = "{:,.2f}".format

# Load and clean data


In [2]:
dir_path = "../../data/exiobase/_ixi/IOT_2014_ixi/"
rexia_dir_path = "../../data/exiobase/REX/rexia-data/converted/"

In [3]:
# Import A, Y and x from Exiobase 3.8
# ! 'A' is a large datasets, mind load time
A = pd.read_csv(f"{dir_path}A.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
# Z = pd.read_csv(f"{dir_path}Z.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
Y = pd.read_csv(f"{dir_path}Y.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
x = pd.read_csv(f"{dir_path}x.txt", delimiter="\t", header=0, index_col=[0, 1])["indout"]
# Import F, F_y (extensions) from both satellite and impacts
F_satellite = pd.read_csv(
    f"{dir_path}satellite/F.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_y_satellite = pd.read_csv(
    f"{dir_path}satellite/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_unit_satellite = pd.read_csv(
    f"{dir_path}satellite/unit.txt", delimiter="\t", header=0, index_col=[0]
)
F_impact = pd.read_csv(f"{dir_path}impacts/F.txt", delimiter="\t", header=[0, 1], index_col=[0])
F_y_impact = pd.read_csv(f"{dir_path}impacts/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0])
F_unit_impact = pd.read_csv(f"{dir_path}impacts/unit.txt", delimiter="\t", header=0, index_col=[0])
# ! Also import Q (extension) from REXIA (Cabernard & Pfister, 2022)
Q = pd.read_csv(f"{rexia_dir_path}Q_REXIA.csv", header=[0, 1], index_col=[0])

In [4]:
# get multiindex
region_and_sector_labels = A.index

# get each level using  'get_level_values()'
region_labels = A.index.get_level_values(level=0).unique()
sector_labels = A.index.get_level_values(level=1).unique()
FD_sector_labels = Y.columns.get_level_values(level=1).unique()

In [5]:
# Some sector labels of Q do not match those of Exiobase
wrong_labels = [
    '"Fishing, operating of fish hatcheries and fish farms; service activities incidental to fishing (05)"',
    '"Manufacture of furniture; manufacturing n.e.c. (36)"',
    '"Manufacture of gas; distribution of gaseous fuels through mains"',
    '"Manufacture of wearing apparel; dressing and dyeing of fur (18)"',
    '"Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials (20)"',
    '"Mining of coal and lignite; extraction of peat (10)"',
    '"Public administration and defence; compulsory social security (75)"',
    '"Retail trade, except of motor vehicles and motorcycles; repair of personal and household goods (52)"',
    '"Supporting and auxiliary transport activities; activities of travel agencies (63)"',
    '"Tanning and dressing of leather; manufacture of luggage, handbags, saddlery, harness and footwear (19)"',
]
correct_labels = [
    "Fishing, operating of fish hatcheries and fish farms; service activities incidental to fishing (05)",
    "Manufacture of furniture; manufacturing n.e.c. (36)",
    "Manufacture of gas; distribution of gaseous fuels through mains",
    "Manufacture of wearing apparel; dressing and dyeing of fur (18)",
    "Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials (20)",
    "Mining of coal and lignite; extraction of peat (10)",
    "Public administration and defence; compulsory social security (75)",
    "Retail trade, except of motor vehicles and motorcycles; repair of personal and household goods (52)",
    "Supporting and auxiliary transport activities; activities of travel agencies (63)",
    "Tanning and dressing of leather; manufacture of luggage, handbags, saddlery, harness and footwear (19)",
]

sectors_mapping = {wrong_labels[i]: correct_labels[i] for i, _ in enumerate(wrong_labels)}
Q.rename(columns=sectors_mapping, inplace=True)

# make sure that both indexes are the same
sorted(sector_labels) == sorted(Q.columns.get_level_values(level=1).unique())

True

In [6]:
# Aggregate countries so as to match the region clusters of Exiobase

# Use the mapping file to determine which countries is associated to which cluster
# And also change to country code instead of the full country name
countries_mapping = pd.read_csv("./countries_mapping.csv", header=None)
countries_mapping.set_index(0, inplace=True)

# check that each country is associated with a region associated in the csv file
for country in list(Q.columns.get_level_values(0).unique()):
    if len(countries_mapping.loc[country, 1]) != 2:
        print(country)

# Assign each country to its region
clusters = dict()
for code in countries_mapping[1]:
    clusters[code] = []
for country in list(Q.columns.get_level_values(0).unique()):
    clusters[countries_mapping.loc[country, 1]].append(country)

# check that we have the right number of clusters
print((len(clusters) == 49) & (sorted(region_labels) == sorted(clusters.keys())))

True


In [7]:
# Update Q with the new region and sector labels
new_Q = []
for code, countries in clusters.items():
    res = Q.loc[:, clusters[code]].groupby(level=1, axis=1).sum().loc[:, sector_labels]
    res = pd.concat([res], keys=[code], axis=1)
    new_Q.append(res)

new_Q = pd.concat(new_Q, axis=1)
new_Q.columns.set_names(["region", "sector"], inplace=True)
new_Q.index.set_names(["stressor"], inplace=True)
del Q
new_Q

region                                                                    AT   
sector                                             Cultivation of paddy rice   
stressor                                                                       
BGS mining data [t]                                                     0.00  \
Land use area of mining ? primary commodity all...                      0.00   
Land-use related biodiversity loss of mining ? ...                      0.00   
Land use area of mining ? monetary allocation [...                      0.00   
Land-use related biodiversity loss of mining ? ...                      0.00   
Total material footprint [kt]                                           0.00   
Climate change impacts [kg CO2-eq]                                      0.00   
Particulate-matter related health impacts [DALYs]                       0.00   
Blue water consumption [Mio. m3]                                        0.00   
Water stress [Mio. m3 H2O-eq]                                           0.00   
Total land use area based on EXIOBASE3 [km2]                            0.00   
Total land-use related biodiversity loss based ...                      0.00   
Total land use area based on FAOSTAT for agricu...                      0.00   
Total land-use related biodiversity loss based ...                      0.00   

region                                                                    
sector                                             Cultivation of wheat   
stressor                                                                  
BGS mining data [t]                                                0.00  \
Land use area of mining ? primary commodity all...                 0.00   
Land-use related biodiversity loss of mining ? ...                 0.00   
Land use area of mining ? monetary allocation [...                 0.00   
Land-use related biodiversity loss of mining ? ...                 0.00   
Total material footprint [kt]                                  1,632.32   
Climate change impacts [kg CO2-eq]                       564,031,974.79   
Particulate-matter related health impacts [DALYs]                237.72   
Blue water consumption [Mio. m3]                                   0.79   
Water stress [Mio. m3 H2O-eq]                                      1.00   
Total land use area based on EXIOBASE3 [km2]                   3,118.04   
Total land-use related biodiversity loss based ...         5,138,529.87   
Total land use area based on FAOSTAT for agricu...           304,645.00   
Total land-use related biodiversity loss based ...         5,250,904.40   

region                                                                                
sector                                             Cultivation of cereal grains nec   
stressor                                                                              
BGS mining data [t]                                                            0.00  \
Land use area of mining ? primary commodity all...                             0.00   
Land-use related biodiversity loss of mining ? ...                             0.00   
Land use area of mining ? monetary allocation [...                             0.00   
Land-use related biodiversity loss of mining ? ...                             0.00   
Total material footprint [kt]                                              3,133.72   
Climate change impacts [kg CO2-eq]                                   818,868,108.21   
Particulate-matter related health impacts [DALYs]                            364.99   
Blue water consumption [Mio. m3]                                               6.86   
Water stress [Mio. m3 H2O-eq]                                                  8.68   
Total land use area based on EXIOBASE3 [km2]                               4,459.47   
Total land-use related biodiversity loss based ...                     8,824,797.53   
Total land use area based on FAOSTAT for agricu...        

# Compute Leontief


In [8]:
# Create an identity matrix the same order
Id = np.identity(len(region_and_sector_labels))

# Compute leontief inverse matrix in the Demand-pull model
L_values = np.linalg.inv((Id - A))
L = pd.DataFrame(data=L_values, index=region_and_sector_labels, columns=region_and_sector_labels)

# compute extension intensity
f_satellite = F_satellite / x.T
f_satellite = f_satellite.replace([np.inf, -np.inf, np.nan], 0)  # ! don't forget to replace

f_impact = F_impact / x.T
f_impact = f_impact.replace([np.inf, -np.inf, np.nan], 0)  # ! don't forget to replace

# compute extension intensity
f_rexia = new_Q / x.T
f_rexia = f_rexia.replace([np.inf, -np.inf, np.nan], 0)  # ! don't forget to replace

# 2) Supply chain of nickel


In [9]:
# Find all the sectors that contain the word 'nickel'
MASK = sector_labels.str.contains("nickel", flags=re.IGNORECASE, regex=True)
idx = sector_labels[MASK]
print(*list(zip(sector_labels.get_indexer_for(idx), idx)), sep="\n")

(26, 'Mining of nickel ores and concentrates')


In [10]:
# Derive index
# ! (matlab code lines 568-610)

NICKEL = sector_labels[np.r_[26]]

index_t = pd.IndexSlice[:, NICKEL]
index_t = A.loc[index_t, :].index  # to have explicit labels instead of slice(None)

index_o = region_and_sector_labels.difference(index_t)

# Show result
index_t[:5]

MultiIndex([('AT', 'Mining of nickel ores and concentrates'),
            ('BE', 'Mining of nickel ores and concentrates'),
            ('BG', 'Mining of nickel ores and concentrates'),
            ('CY', 'Mining of nickel ores and concentrates'),
            ('CZ', 'Mining of nickel ores and concentrates')],
           names=['region', 'sector'])

In [11]:
# Derive the Leontief inverse for the remaining economy
# ! (matlab code lines 612 - 615)

# Create an identity matrix the same order
Id = np.identity(len(index_o))

# get corresponding (numerical) index for index_o
ix_o = np.sort(region_and_sector_labels.get_indexer_for(index_o))

# Compute leontief inverse matrix in the Demand-pull model (for the remaining economy )
L_oo_dash = pd.DataFrame(
    data=np.zeros(shape=A.shape),
    index=region_and_sector_labels,
    columns=region_and_sector_labels,
)
L_oo_dash.iloc[ix_o, ix_o] = np.linalg.inv(Id - A.iloc[ix_o, ix_o])

# save intermediary calculation that will be reused multiple times
Y_region = Y.groupby(by="region", axis=1).sum()
Y_region = Y_region.loc[region_and_sector_labels, region_labels]  # reorder rows / cols
Y_global = Y_region.sum(axis=1)
AtoLoo = A.loc[index_t, index_o] @ L_oo_dash.loc[index_o, index_o]

In [12]:
# Show results
Y_region

region                                                        AT   BE   BG   
region sector                                                                
AT     Cultivation of paddy rice                            0.00 0.00 0.00  \
       Cultivation of wheat                                 0.00 0.10 2.20   
       Cultivation of cereal grains nec                   152.07 0.32 0.37   
       Cultivation of vegetables, fruit, nuts             803.64 0.33 1.37   
       Cultivation of oil seeds                             0.00 0.14 3.48   
...                                                          ...  ...  ...   
WM     Activities of membership organisation n.e.c. (91)    1.06 3.42 2.26   
       Recreational, cultural and sporting activities ...   9.15 2.20 3.83   
       Other service activities (93)                        1.54 3.59 0.14   
       Private households with employed persons (95)        0.17 1.36 0.02   
       Extra-territorial organizations and bodies           0.00 0.00 0.00   

region                                                      CY   CZ    DE   
region sector                                                               
AT     Cultivation of paddy rice                          0.00 0.00  0.00  \
       Cultivation of wheat                               0.01 0.21 14.69   
       Cultivation of cereal grains nec                   0.02 0.50 13.44   
       Cultivation of vegetables, fruit, nuts             0.09 6.68 98.67   
       Cultivation of oil seeds                           0.00 0.52 24.16   
...                                                        ...  ...   ...   
WM     Activities of membership organisation n.e.c. (91)  0.23 0.51  5.08   
       Recreational, cultural and sporting activities ... 3.69 8.63 13.57   
       Other service activities (93)                      0.01 0.39  2.65   
       Private households with employed persons (95)      0.16 0.10  1.41   
       Extra-territorial organizations and bodies         0.00 0.00  0.00   

region                                                       DK   EE    ES   
region sector                                                                
AT     Cultivation of paddy rice                           0.00 0.00  0.00  \
       Cultivation of wheat                                0.00 0.01  0.28   
       Cultivation of cereal grains nec                    0.01 0.03  0.15   
       Cultivation of vegetables, fruit, nuts              0.60 0.03  4.29   
       Cultivation of oil seeds                            0.83 0.00  0.03   
...                                                         ...  ...   ...   
WM     Activities of membership organisation n.e.c. (91)   2.78 0.37  4.85   
       Recreational, cultural and sporting activities ... 14.07 0.29 20.33   
       Other service activities (93)                       1.06 0.02  5.36   
       Private households with employed persons (95)       0.07 0.02 90.87   
       Extra-territorial organizations and bodies          0.00 0.00  0.00   

region                                                       FI  ...   TR   
region sector                                                    ...        
AT     Cultivation of paddy rice                           0.00  ... 0.00  \
       Cultivation of wheat                                0.01  ... 0.01   
       Cultivation of cereal grains nec                    0.03  ... 0.03   
       Cultivation of vegetables, fruit, nuts              1.28  ... 0.48   
       Cultivation of oil seeds                            0.16  ... 0.00   
...                                                         ...  ...  ...   
WM     Activities of membership organisation n.e.c. (91)   1.29  ... 1.09   
       Recreational, cultural and sporting activities ... 10.59  ... 0.12   
       Other service activities (93)                       1.64  ... 3.62   
       Private households with employed persons (95)       0.10  ... 1.00   
       Extra-territorial organizations and b

In [13]:
# Show results
Y_global

region  sector                                             
AT      Cultivation of paddy rice                                  0.00
        Cultivation of wheat                                     112.16
        Cultivation of cereal grains nec                         192.44
        Cultivation of vegetables, fruit, nuts                 1,033.94
        Cultivation of oil seeds                                  41.69
                                                                 ...   
WM      Activities of membership organisation n.e.c. (91)      9,206.81
        Recreational, cultural and sporting activities (92)   29,868.39
        Other service activities (93)                         18,556.71
        Private households with employed persons (95)          4,022.69
        Extra-territorial organizations and bodies                 0.00
Length: 7987, dtype: float64

## Choice of stressor


In [14]:
# Select stressors to be analysed
# ! (matlab code line ....)

# # Option 1: if we precisely know which stressor we want to analyse
# idx = [
#     "Domestic Extraction Used - Metal Ores - Nickel ores",
#     "Unused Domestic Extraction - Metal Ores - Nickel ores",
# ]

# # Option 2: find all the stressors that contain the word...
# stressor = "Water Withdrawal Blue - Manufacturing"
# MASK = d.index.str.contains(stressor, flags=re.IGNORECASE, regex=True)
# idx = d.loc[MASK].index
# print(*list(zip(d.index.get_indexer_for(idx), idx)), sep="\n")

# Option 3: use Q from REXIA (Cabernard & Pfister, 2022)
d = f_rexia  # choose f_impact / f_satellite / f_biodiv
stressor_labels = d.index  # .get_level_values(level=1).unique()
list(enumerate(d.index))

[(0, 'BGS mining data [t]'),
 (1, 'Land use area of mining ? primary commodity allocation [km2]'),
 (2,
  'Land-use related biodiversity loss of mining ? primary commodity allocation [global pdf]'),
 (3, 'Land use area of mining ? monetary allocation [km2]'),
 (4,
  'Land-use related biodiversity loss of mining ? monetary allocation [global pdf]'),
 (5, 'Total material footprint [kt]'),
 (6, 'Climate change impacts [kg CO2-eq]'),
 (7, 'Particulate-matter related health impacts [DALYs]'),
 (8, 'Blue water consumption [Mio. m3]'),
 (9, 'Water stress [Mio. m3 H2O-eq]'),
 (10, 'Total land use area based on EXIOBASE3 [km2]'),
 (11,
  'Total land-use related biodiversity loss based on EXIOBASE3 [10^12 global pdf]'),
 (12,
  'Total land use area based on FAOSTAT for agriculture and forestry [km2]'),
 (13,
  'Total land-use related biodiversity loss based on FAOSTAT for agriculture and forestry [10^12 global pdf]')]

In [63]:
d_i = d.iloc[np.r_[4]].sum()
stress = "biodiv"  # for the sankey (excel) data

# save intermediary calculation that will be reused multiple times
dLt = d_i @ L.loc[:, index_t]

# # ! make sure it is a column vector (= pd.Series). use sum() if necessary
d_i

region  sector                                             
AT      Cultivation of paddy rice                             0.00
        Cultivation of wheat                                  0.00
        Cultivation of cereal grains nec                      0.00
        Cultivation of vegetables, fruit, nuts                0.00
        Cultivation of oil seeds                              0.00
                                                              ... 
WM      Activities of membership organisation n.e.c. (91)     0.00
        Recreational, cultural and sporting activities (92)   0.00
        Other service activities (93)                         0.00
        Private households with employed persons (95)         0.00
        Extra-territorial organizations and bodies            0.00
Name: Land-use related biodiversity loss of mining ? monetary allocation [global pdf], Length: 7987, dtype: float64

## Link Preg Treg

$$E_{P,T} = diag\biggl(d_{i}\biggr) \times L_{all, T} \times diag\biggl(Y_{all, T} + A_{T,O} \times L'_{O,O} \times Y_{all, O}\biggr)$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [64]:
# ! (matlab code lines 618-625)

E_P_T = pd.DataFrame(
    data=np.zeros(shape=A.shape),
    index=region_and_sector_labels,
    columns=region_and_sector_labels,
)

# Compute Lall-t . diag( Yall-t +  Ato.Loo.Yall-o)
Lt_x_Yt_plus_AtoLooY_GLO = (
    L.loc[:, index_t] * (Y_global.loc[index_t] + AtoLoo @ Y_global.loc[index_o]).T
)

# Compute diag(di) . Lall-t . diag( Yall-t +  Ato.Loo.Yall-o)
E_P_T.loc[:, index_t] = (d_i * Lt_x_Yt_plus_AtoLooY_GLO.T).T.values

# sum the sectors together and keep regions disaggregated along both axis
E_Preg_Treg = E_P_T.groupby("region", axis=0).sum().groupby("region", axis=1).sum()

# reorder rows and cols
E_Preg_Treg = E_Preg_Treg.loc[region_labels, region_labels]

# delete variables that are no longer used to save memory
del Lt_x_Yt_plus_AtoLooY_GLO

# show results
E_Preg_Treg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
region,,,,,,,,,,,,,,,,,,,,,
AT,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,...,0.00,0.00,0.00,0.15,0.01,0.37,0.04,0.02,0.10,0.05
BE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BG,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.11,0.06,...,0.01,0.00,0.00,0.93,0.05,3.44,0.26,0.31,0.52,0.03
CY,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.01,...,0.01,0.00,0.00,0.70,0.01,0.81,0.12,0.04,0.15,0.01
CZ,0.00,0.00,0.00,0.00,3.14,0.00,0.00,0.00,0.03,0.00,...,0.00,0.00,0.00,0.17,0.01,0.29,0.09,0.16,0.16,0.01
DE,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.14,0.02,...,0.01,0.00,0.01,0.45,0.07,0.94,0.30,0.23,0.59,0.04
DK,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
EE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"40,607.40",0.01,...,0.01,0.00,0.00,1.41,0.06,3.78,0.55,0.42,1.09,0.05


## Link Treg FDreg

$$E_{T, FDreg} = diag\biggl(d_{i} \times L_{all, T}\biggr) \times \biggl(Y_{T, reg} + A_{T,O} \times L'_{O,O} \times Y_{O, reg}\biggr)$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [65]:
# ! (matlab code lines 627-629)

E_T_FDreg = pd.DataFrame(
    data=np.zeros(shape=(len(region_and_sector_labels), len(region_labels))),
    index=region_and_sector_labels,
    columns=region_labels,
)

# Compute Yrt + Ato.Loo.Yro
Yt_plus_AtoLooY_REG = Y_region.loc[index_t, :] + AtoLoo @ Y_region.loc[index_o, :]

# Compute diag(di) .  Lall-t  .  ( Yrt + Ato.Loo.Yro )
E_T_FDreg.loc[index_t, :] = (dLt * Yt_plus_AtoLooY_REG.T).T.values

# group sectors together (along the rows) and keep regions disaggregated
E_Treg_FDreg = E_T_FDreg.groupby("region", axis=0).sum().loc[region_labels]

# reorder rows and cols
E_Treg_FDreg = E_Treg_FDreg.loc[region_labels, region_labels]

# delete variables that are no longer used to save memory space
del E_T_FDreg

# Show results
E_Treg_FDreg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
region,,,,,,,,,,,,,,,,,,,,,
AT,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BG,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
CY,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
CZ,0.09,0.05,0.01,0.00,1.03,0.74,0.02,0.00,0.07,0.02,...,0.08,0.01,0.03,0.01,0.01,0.14,0.06,0.03,0.05,0.14
DE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
DK,0.05,0.06,0.01,0.00,0.04,0.62,2.55,0.02,0.09,0.09,...,0.09,0.02,0.41,0.02,0.02,0.29,0.18,0.05,0.12,0.27
EE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ES,47.13,117.39,13.90,4.15,39.38,521.87,34.69,6.19,998.61,34.11,...,229.30,"1,423.02",65.85,466.95,66.62,"3,648.49","1,203.44",76.51,977.03,"1,810.86"


## Link Preg FDreg

$$E_{P, FDreg} = diag(d_{i}) \times L_{all, T} \times (Y_{T, reg} + A_{T,O} \times L'_{O,O} \times Y_{O, reg})$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [66]:
# ! (matlab code lines 635-640)

# Compute  Lgt . ( Yrt + Ato.Loo.Yro )
Lt_dot_Yt_plus_AtoLooY_REG = L.loc[:, index_t] @ Yt_plus_AtoLooY_REG

# Compute  diag(di) .  Lall-t . ( Yrt + Ato.Loo.Yro )
E_P_FDreg = (d_i * Lt_dot_Yt_plus_AtoLooY_REG.T).T

# group sectors together (along the rows) and keep regions disaggregated
E_Preg_FDreg = E_P_FDreg.groupby("region", axis=0).sum()

# reorder rows and cols
E_Preg_FDreg = E_Preg_FDreg.loc[region_labels, region_labels]

# delete variables that are no longer used to save memory space
del Yt_plus_AtoLooY_REG, Lt_dot_Yt_plus_AtoLooY_REG, E_P_FDreg

# show results
E_Preg_FDreg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
region,,,,,,,,,,,,,,,,,,,,,
AT,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.01,0.00,...,0.01,0.01,0.00,0.10,0.01,0.12,0.04,0.03,0.07,0.05
BE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BG,0.27,0.55,0.06,0.02,0.22,2.82,0.16,0.03,0.72,0.20,...,1.07,0.28,0.31,1.99,0.24,20.12,4.25,0.67,3.51,6.59
CY,0.00,0.01,0.00,0.00,0.00,0.03,0.00,0.00,0.01,0.01,...,0.01,0.02,0.01,0.44,0.01,0.22,0.08,0.07,0.10,0.07
CZ,0.07,0.04,0.01,0.00,0.72,0.54,0.01,0.00,0.06,0.02,...,0.06,0.02,0.02,0.12,0.01,0.25,0.11,0.19,0.14,0.16
DE,0.01,0.02,0.00,0.00,0.01,0.08,0.01,0.00,0.03,0.02,...,0.03,0.03,0.01,0.31,0.04,0.47,0.22,0.28,0.38,0.17
DK,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
EE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ES,47.60,118.27,13.99,4.17,39.76,526.54,34.93,6.24,997.99,34.42,...,231.00,"1,420.56",66.34,469.71,66.97,"3,681.82","1,209.52",77.51,982.17,"1,820.50"


## Link Tsec Psec


In [67]:
# ! (matlab code lines 642-651)

# group E_P_T regions together (along rows + columns) and keep sectors disaggregated
E_Psec_Tsec = E_P_T.groupby("sector", axis=0).sum().groupby("sector", axis=1).sum()

# reorder cols/rows and transpose
E_Psec_Tsec = E_Psec_Tsec.loc[sector_labels, sector_labels]
E_Tsec_Psec = E_Psec_Tsec.T

# delete variables that are no longer used to save memory space
del E_Psec_Tsec

# show result
E_Tsec_Psec

sector,Cultivation of paddy rice,Cultivation of wheat,Cultivation of cereal grains nec,"Cultivation of vegetables, fruit, nuts",Cultivation of oil seeds,"Cultivation of sugar cane, sugar beet",Cultivation of plant-based fibers,Cultivation of crops nec,Cattle farming,Pigs farming,...,Landfill of waste: Paper,Landfill of waste: Plastic,Landfill of waste: Inert/metal/hazardous,Landfill of waste: Textiles,Landfill of waste: Wood,Activities of membership organisation n.e.c. (91),"Recreational, cultural and sporting activities (92)",Other service activities (93),Private households with employed persons (95),Extra-territorial organizations and bodies
sector,,,,,,,,,,,,,,,,,,,,,
Cultivation of paddy rice,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of wheat,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of cereal grains nec,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Cultivation of vegetables, fruit, nuts",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of oil seeds,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Recreational, cultural and sporting activities (92)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other service activities (93),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Link Psec Preg


In [68]:
# ! (matlab code lines 653-655)

# sum the columns of E_P_T and reshape the vector into 163 x 49
E_Psec_Preg = E_P_T.sum(axis=1).unstack(level=0)

# reorder cols/rows
E_Psec_Preg = E_Psec_Preg.loc[sector_labels, region_labels]

# show result
E_Psec_Preg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
sector,,,,,,,,,,,,,,,,,,,,,
Cultivation of paddy rice,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of wheat,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of cereal grains nec,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Cultivation of vegetables, fruit, nuts",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of oil seeds,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Recreational, cultural and sporting activities (92)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other service activities (93),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## link FDreg FSsec

$$
\begin{align}
    E_{FS, FDreg} &= diag\biggl(d_{i} \times L_{all, T}\biggr) \times Y_{T, reg}\\
    &+ diag\biggl(d_{i} \times L_{all, T} \times A_{T,O} \times L'_{O,O}\biggr) \times Y_{O, reg}\\
\end{align}
$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [69]:
# ! (matlab code lines 658-670)

# Compute diag(di . Lall-t) . Yrt
E_FS_FDreg_t = (dLt * Y_region.loc[index_t, :].T).T

# reorder cols/rows
E_FS_FDreg_t = E_FS_FDreg_t.loc[index_t, region_labels]

# Compute diag(di . Lt . Ato . Loo)  .  Yro
E_FS_FDreg_o = ((dLt @ AtoLoo) * Y_region.loc[index_o, :].T).T

# reorder cols/rows
E_FS_FDreg_o = E_FS_FDreg_o.loc[index_o, region_labels]

# join both matrices to get the full E_FS_FDreg
E_FS_FDreg = pd.concat([E_FS_FDreg_t, E_FS_FDreg_o], axis=0)

# reorder cols/rows
E_FS_FDreg = E_FS_FDreg.loc[region_and_sector_labels, region_labels]

# group regions together (along rows) and keep sectors disaggregated
E_FSsec_FDreg = E_FS_FDreg.groupby("sector", axis=0).sum()

# reorder cols/rows and transpose
E_FSsec_FDreg = E_FSsec_FDreg.loc[sector_labels, region_labels]
E_FDreg_FSsec = E_FSsec_FDreg.T

# save space by removing variables that are not used anymore
# del E_FS_FDreg_t, E_FS_FDreg_o, E_FSsec_FDreg, E_FS_FDreg

# show result
E_FDreg_FSsec

sector,Cultivation of paddy rice,Cultivation of wheat,Cultivation of cereal grains nec,"Cultivation of vegetables, fruit, nuts",Cultivation of oil seeds,"Cultivation of sugar cane, sugar beet",Cultivation of plant-based fibers,Cultivation of crops nec,Cattle farming,Pigs farming,...,Landfill of waste: Paper,Landfill of waste: Plastic,Landfill of waste: Inert/metal/hazardous,Landfill of waste: Textiles,Landfill of waste: Wood,Activities of membership organisation n.e.c. (91),"Recreational, cultural and sporting activities (92)",Other service activities (93),Private households with employed persons (95),Extra-territorial organizations and bodies
region,,,,,,,,,,,,,,,,,,,,,
AT,0.59,15.35,37.82,197.87,21.81,0.42,3.22,3.99,27.87,28.09,...,5.12,2.34,5.78,3.35,21.42,305.92,340.42,150.05,0.50,0.00
BE,17.61,62.73,12.12,366.96,15.82,2.08,29.62,283.35,41.71,35.33,...,16.91,9.60,19.06,6.44,9.47,108.18,970.32,865.57,1.79,0.00
BG,1.74,4.07,7.90,75.30,18.74,0.13,2.35,44.22,1.02,0.73,...,3.87,1.53,3.76,0.64,0.92,6.88,138.60,17.93,0.07,0.00
CY,0.32,1.76,1.48,14.16,0.38,0.13,0.51,5.38,0.48,0.57,...,3.34,3.09,2.57,0.76,1.17,9.36,53.27,9.91,0.17,0.00
CZ,2.30,81.26,28.82,135.34,31.98,0.69,8.45,55.34,4.73,19.50,...,25.77,3.98,16.07,5.04,6.44,117.96,531.74,98.28,0.20,0.00
DE,79.91,310.05,144.14,"1,785.13",131.80,12.16,87.87,"1,238.86",89.94,124.95,...,478.71,124.07,500.74,64.13,115.24,"1,452.97","2,747.15","1,374.53",4.21,0.00
DK,1.42,167.07,54.02,86.49,17.85,0.39,2.74,25.63,13.56,19.57,...,7.16,5.18,18.64,2.92,4.16,212.57,643.91,114.76,0.54,0.00
EE,0.77,0.75,5.87,17.06,0.31,0.15,1.77,28.04,3.89,5.34,...,2.22,1.29,1.81,0.46,0.63,8.12,36.05,20.21,0.22,0.00
ES,36.77,158.50,24.54,850.70,8.87,1.82,13.00,447.81,84.68,152.78,...,132.39,43.08,184.74,28.43,41.56,579.29,"2,096.45",981.19,504.42,0.00


## Link Tsec Treg


In [70]:
# ! (matlab code lines 672-674)

# sum the rows of E_P_T and reshape the vector into 163 x 49
E_Tsec_Treg = E_P_T.sum(axis=0).unstack(level=0)

# reorder cols/rows
E_Tsec_Treg = E_Tsec_Treg.loc[sector_labels, region_labels]

# show result
E_Tsec_Treg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
sector,,,,,,,,,,,,,,,,,,,,,
Cultivation of paddy rice,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of wheat,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of cereal grains nec,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Cultivation of vegetables, fruit, nuts",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of oil seeds,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Recreational, cultural and sporting activities (92)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other service activities (93),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## link Preg vs FSsec

This section is extra compared to the Nickel analysis

$$
\begin{align}
    E_{P, FS} &= diag\biggl(d_{i}\biggr)  \times L_{all, T}\times diag\biggl(Y_{all, T} \biggr)\\
    &+ diag\biggl(d_{i}\biggr) \times L_{all, T} \times A_{T,O} \times L'_{O,O} \times  diag\biggl(Y_{all, O} \biggr)\\
\end{align}
$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [71]:
# ! (matlab code lines 857-869)

# Compute L . diag(Yt)
diagLt_Yt = L.loc[:, index_t] * Y_global.loc[index_t].T

# Compute diag(di) . L . diag(Yt)
E_P_FS_t = (d_i * diagLt_Yt.T).T

# Compute  (L Ato Loo) . diag(Yo)
LtAtoLoo_diagYo = (L.loc[:, index_t] @ AtoLoo) * Y_global.loc[index_o].T

# Compute diag(di) . (L Ato Loo) . diag(Yo)
E_P_FS_o = (d_i * LtAtoLoo_diagYo.T).T

# join both matrices to get full E_FS_FDreg
E_P_FS = pd.concat([E_P_FS_t, E_P_FS_o], axis=1)

# reorder cols/rows
E_P_FS = E_P_FS.loc[region_and_sector_labels, region_labels]

# group E_P_FS regions together (along rows + columns) and keep sectors disaggregated
E_Psec_FSsec = E_P_FS.groupby("sector", axis=0).sum().groupby("sector", axis=1).sum()

# reorder cols/rows
E_Psec_FSsec = E_Psec_FSsec.loc[sector_labels, sector_labels]

# save space by removing variables that are not used anymore
# del diagLt_Yt, LtAtoLoo_diagYo, E_P_FS_t, E_P_FS_o, E_P_FS

# show result
E_Psec_FSsec

sector,Cultivation of paddy rice,Cultivation of wheat,Cultivation of cereal grains nec,"Cultivation of vegetables, fruit, nuts",Cultivation of oil seeds,"Cultivation of sugar cane, sugar beet",Cultivation of plant-based fibers,Cultivation of crops nec,Cattle farming,Pigs farming,...,Landfill of waste: Paper,Landfill of waste: Plastic,Landfill of waste: Inert/metal/hazardous,Landfill of waste: Textiles,Landfill of waste: Wood,Activities of membership organisation n.e.c. (91),"Recreational, cultural and sporting activities (92)",Other service activities (93),Private households with employed persons (95),Extra-territorial organizations and bodies
sector,,,,,,,,,,,,,,,,,,,,,
Cultivation of paddy rice,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of wheat,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of cereal grains nec,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Cultivation of vegetables, fruit, nuts",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of oil seeds,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Recreational, cultural and sporting activities (92)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other service activities (93),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


# Sankeys preparation


## Preg


In [72]:
# ! (matlab code lines 932-944)

agg_regions = {
    "Australia": region_labels == "AU",
    # "Belgium": region_labels == "BE",
    "Brazil": region_labels == "BR",
    # "Bulgaria": region_labels == "BG",
    # "Canada": region_labels == "CA",
    "China": region_labels == "CN",
    # "Germany": region_labels == "DE",
    # "India": region_labels == "IN",
    "Indonesia": region_labels == "ID",
    "Japan": region_labels == "JP",
    # "Mexico": region_labels == "MX",
    # "Netherlands": region_labels == "NL",
    # "Poland": region_labels == "PL",
    # "South Korea": region_labels == "KR",
    # "Russia": region_labels == "RU",
    # "Spain": region_labels == "ES",
    "USA": region_labels == "US",
    "EU27": region_labels.isin(region_labels[:28]),
    # "RoW Asia Pacific": (region_labels == "WA") | (region_labels == "TW"),
    # "RoW America": region_labels == "WL",
    # "RoW Africa": (region_labels == "WF") | (region_labels == "ZA"),
    # "RoW Middle East": region_labels == "WM",
}
# agg_regions["RoW Europe"] = ~reduce(np.logical_or, list(agg_regions.values()))
agg_regions["other"] = ~reduce(np.logical_or, list(agg_regions.values()))

Agg_Preg = pd.DataFrame(index=region_labels, columns=agg_regions.keys())
for regions in Agg_Preg.columns:
    Agg_Preg.loc[:, regions] = agg_regions[regions].astype(int)

if sum(sum(agg_regions.values())) != len(region_labels):
    print("Warning: some regions are missing in the aggregation matrix")

Agg_Preg

,Australia,Brazil,China,Indonesia,Japan,USA,EU27,other
region,,,,,,,,
AT,0,0,0,0,0,0,1,0
BE,0,0,0,0,0,0,1,0
BG,0,0,0,0,0,0,1,0
CY,0,0,0,0,0,0,1,0
CZ,0,0,0,0,0,0,1,0
DE,0,0,0,0,0,0,1,0
DK,0,0,0,0,0,0,1,0
EE,0,0,0,0,0,0,1,0
ES,0,0,0,0,0,0,1,0


## FSsec


In [73]:
# ! (matlab code lines 947-963)

agg_sectors = {
    "agrifood": sector_labels[np.r_[0:19, 34:46]],
    "mining": sector_labels[19:34],
    "manufacturing": sector_labels[np.r_[46:85, 92]],
    "machinery_electronics": sector_labels[85:90],
    "auto": sector_labels[90:92],
    "electricity": sector_labels[95:109],
    "gas_water": sector_labels[109:112],
    "construction": sector_labels[112:114],
    "service": sector_labels[np.r_[114:119, 125:138]],
    "transportation": sector_labels[119:125],
    "waste": sector_labels[np.r_[93:95, 138:158]],
    "other_sec": sector_labels[158:],
}

Agg_FSsec = pd.DataFrame(index=sector_labels, columns=agg_sectors.keys())
for sectors in Agg_FSsec.columns:
    Agg_FSsec.loc[:, sectors] = sector_labels.isin(agg_sectors[sectors]).astype(int)

if sum(len(sec) for sec in agg_sectors.values()) != len(sector_labels):
    print("Warning: some sectors are missing in the aggregation matrix")

Agg_FSsec

,agrifood,mining,manufacturing,machinery_electronics,auto,electricity,gas_water,construction,service,transportation,waste,other_sec
sector,,,,,,,,,,,,
Cultivation of paddy rice,1,0,0,0,0,0,0,0,0,0,0,0
Cultivation of wheat,1,0,0,0,0,0,0,0,0,0,0,0
Cultivation of cereal grains nec,1,0,0,0,0,0,0,0,0,0,0,0
"Cultivation of vegetables, fruit, nuts",1,0,0,0,0,0,0,0,0,0,0,0
Cultivation of oil seeds,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0,0,0,0,0,0,0,0,0,0,0,1
"Recreational, cultural and sporting activities (92)",0,0,0,0,0,0,0,0,0,0,0,1
Other service activities (93),0,0,0,0,0,0,0,0,0,0,0,1


## Psec


In [74]:
# ! (matlab code lines 886-906)

agg_production = {
    "coal": sector_labels[[19]],
    "nickel": sector_labels[[26]],
    "iron": sector_labels[np.r_[24, 71, 72]],
    "copper": sector_labels[np.r_[25, 79, 80]],
    "aluminium": sector_labels[np.r_[27, 75, 76]],
    "precious_metals": sector_labels[np.r_[28, 73, 74]],
}
all_metals = sector_labels[np.r_[23:31, 70:83]]
primary_metals = (
    set(agg_production["nickel"])
    | set(agg_production["iron"])
    | set(agg_production["copper"])
    | set(agg_production["aluminium"])
    | set(agg_production["precious_metals"])
)
agg_production["other_metals"] = all_metals.difference(primary_metals)
agg_production["non_mining"] = sector_labels.difference(
    set(all_metals) | set(agg_production["coal"])
)

Agg_Psec = pd.DataFrame(index=sector_labels, columns=agg_production.keys())
for sectors in Agg_Psec.columns:
    Agg_Psec.loc[:, sectors] = sector_labels.isin(agg_production[sectors]).astype(int)

# Show
Agg_Psec

,coal,nickel,iron,copper,aluminium,precious_metals,other_metals,non_mining
sector,,,,,,,,
Cultivation of paddy rice,0,0,0,0,0,0,0,1
Cultivation of wheat,0,0,0,0,0,0,0,1
Cultivation of cereal grains nec,0,0,0,0,0,0,0,1
"Cultivation of vegetables, fruit, nuts",0,0,0,0,0,0,0,1
Cultivation of oil seeds,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0,0,0,0,0,0,0,1
"Recreational, cultural and sporting activities (92)",0,0,0,0,0,0,0,1
Other service activities (93),0,0,0,0,0,0,0,1


## Tsec


In [75]:
# ! (matlab code lines ...)

# Remove the "coal" and "non_mining" columns to get Agg_Tsec
Agg_Tsec = Agg_Psec[["nickel", "iron", "copper", "aluminium", "precious_metals", "other_metals"]]
Agg_Tsec

,nickel,iron,copper,aluminium,precious_metals,other_metals
sector,,,,,,
Cultivation of paddy rice,0,0,0,0,0,0
Cultivation of wheat,0,0,0,0,0,0
Cultivation of cereal grains nec,0,0,0,0,0,0
"Cultivation of vegetables, fruit, nuts",0,0,0,0,0,0
Cultivation of oil seeds,0,0,0,0,0,0
...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0,0,0,0,0,0
"Recreational, cultural and sporting activities (92)",0,0,0,0,0,0
Other service activities (93),0,0,0,0,0,0


## Sankey #1


In [76]:
# ! (matlab code lines 965-982)

# shares_Sankey_Tsec_Psec = (Agg_Psec.T @ E_Tsec_Psec.T @ Agg_Tsec) / E_Preg_Treg.sum().sum()
# shares_Sankey_Psec_Preg = (Agg_Psec.T @ E_Psec_Preg @ Agg_Preg) / E_Preg_Treg.sum().sum()
shares_Sankey_Preg_Treg = Agg_Preg.T @ E_Preg_Treg @ Agg_Preg  # / E_Preg_Treg.sum().sum()
shares_Sankey_Treg_FDreg = Agg_Preg.T @ E_Treg_FDreg @ Agg_Preg  # / E_Treg_FDreg.sum().sum()
shares_Sankey_FDreg_FSsec = Agg_Preg.T @ E_FDreg_FSsec @ Agg_FSsec  # / E_FDreg_FSsec.sum().sum()

perspectives = [
    shares_Sankey_Preg_Treg.sum(axis=1),
    shares_Sankey_Treg_FDreg.sum(axis=1),
    shares_Sankey_Treg_FDreg.sum().T,
    shares_Sankey_FDreg_FSsec.sum().T,
]
linkages = [
    shares_Sankey_Preg_Treg.stack(),
    shares_Sankey_Treg_FDreg.stack(),
    shares_Sankey_FDreg_FSsec.stack(),
]

# Show results
threshold = 0.001
print(stress, "\n")
for i in range(3):
    print(f"linkage {i+1}")
    mask = linkages[i] > threshold
    print(linkages[i][mask], "\n\n")
for i in range(4):
    print(f"perspectives {i+1}")
    mask = perspectives[i] > threshold
    print(perspectives[i][mask], "\n\n")

biodiv 

linkage 1
Australia  Australia      749,962.09
           Brazil             184.90
           China            9,334.31
           Indonesia        1,573.10
           EU27                 7.91
           other            1,320.67
Brazil     Australia           30.00
           Brazil         183,089.12
           China              714.49
           Indonesia          323.94
           EU27                 3.25
           other              197.03
China      Australia           21.95
           Brazil              22.56
           China           41,171.50
           Indonesia           38.84
           EU27                 0.79
           other              138.80
Indonesia  Australia          252.01
           Brazil             103.32
           China            6,488.11
           Indonesia    2,571,098.82
           EU27                12.40
           other           11,553.17
Japan      Australia            0.14
           Brazil               0.05
           China   

In [77]:
# Save results
metal = "nickel"
year = 2014
filename = f"{metal}_{stress}_{year}_sankey_1"
mapping = {"level_0": "source_country", "level_1": "target_country", 0: "value"}
stages = ["mining", "processing", "consumption", "End-use"]

for i, _ in enumerate(linkages):
    linkages[i] = linkages[i].reset_index().rename(columns=mapping)
    linkages[i]["source_stage"] = stages[i]
    linkages[i]["target_stage"] = stages[i + 1]
    linkages[i] = linkages[i][
        ["source_stage", "source_country", "target_stage", "target_country", "value"]
    ]

pd.concat(linkages, axis=0).to_csv(f"./results/{metal}/{filename}.csv", index=False)

### Biodiversity

![S25 original](./img/nickel/S25.png)
![S25 reproduced](./img/nickel/S25_reproduced.png)

### Land use

![Land use](./img/nickel/nickel_landuse.png)

### Greenhouse gases

![ghg](./img/nickel/nickel_ghg.png)

### Blue water consumption

![water](./img/nickel/nickel_water.png)
